In [1]:
import numpy as np 
import torch
from torchsummary import summary
import torch.optim as optim
import matplotlib.pyplot as plt
import pandas as pd
import torch; torch.manual_seed(0)
import torch.nn as nn
import torch.nn.functional as F
import torch.utils
import matplotlib.pyplot as plt; plt.rcParams['figure.dpi'] = 200
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [2]:
device = 'cpu'
gen_dir = 'generated_data/'
models_dir = 'models/'

In [12]:
embedding_dim = 16
num_embeddings = 128

In [5]:
lstm_data_df = pd.read_csv(gen_dir+'encoding_indices_data.csv')
print(lstm_data_df.shape)
lstm_data_df.head()

(34286, 64)


,0,1,2,3,4,5,6,7,8,9,...,54,55,56,57,58,59,60,61,62,63
0,88,56,15,81,15,1,1,75,26,116,...,2,29,12,28,88,15,92,120,47,24
1,7,51,60,108,70,70,114,102,22,90,...,45,69,8,90,123,55,90,7,36,76
2,76,8,116,88,20,51,102,20,20,104,...,67,36,81,90,82,115,23,113,101,70
3,53,8,32,34,96,62,20,69,51,35,...,115,34,18,35,82,115,70,95,64,70
4,46,53,116,60,77,35,62,20,13,87,...,30,7,18,23,82,55,18,64,36,62


In [6]:
lstm_data = lstm_data_df.to_numpy()
lstm_data

array([[ 88,  56,  15, ..., 120,  47,  24],
       [  7,  51,  60, ...,   7,  36,  76],
       [ 76,   8, 116, ..., 113, 101,  70],
       ...,
       [ 35, 126, 115, ...,  70,  35, 118],
       [ 36,  23, 114, ...,  24, 108,  20],
       [ 47,  79,  15, ..., 124,  84,   8]])

# LSTM model

In [8]:
seq_length = int(lstm_data.shape[1]/2)
dataX = []
dataY = []
for j in lstm_data:
    for i in range(0, lstm_data.shape[1] - seq_length, 1):
        seq_in = j[i:i + seq_length]
        seq_out = j[i + seq_length]
        dataX.append( seq_in)
        dataY.append(seq_out)
n_patterns = len(dataX)
print("Total Patterns: ", n_patterns)

Total Patterns:  1097152


In [9]:
X = torch.tensor(dataX, dtype=torch.float32).reshape(n_patterns, seq_length, 1)
# X = X / float(num_embeddings)
y = torch.tensor(dataY)
print(X.shape, y.shape)

/var/folders/4t/55hr34s50pb0bz56v_nxv0j00000gn/T/ipykernel_96531/1186672099.py:1: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /Users/runner/work/_temp/anaconda/conda-bld/pytorch_1681196928688/work/torch/csrc/utils/tensor_new.cpp:248.)
  X = torch.tensor(dataX, dtype=torch.float32).reshape(n_patterns, seq_length, 1)


torch.Size([1097152, 32, 1]) torch.Size([1097152])


In [13]:
X = X / float(num_embeddings)
X_train, X_test, y_train, y_test  = train_test_split(X,y, test_size=0.2, random_state=42)

In [14]:

import torch.nn as nn
import torch.optim as optim
import torch.utils.data as datatools

class LSTM_Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.lstm = nn.LSTM(input_size=1, hidden_size=256, num_layers=2, batch_first=True)
        self.dropout = nn.Dropout(0.2)
        self.linear = nn.Linear(256, num_embeddings)
    def forward(self, x):
        x, _ = self.lstm(x)
        # take only the last output
        x = x[:, -1, :]
        # produce output
        x = self.linear(self.dropout(x))
        return x

In [15]:
device = 'mps'
n_epochs = 40
batch_size = 128
lstm_model = LSTM_Model().to(device)

optimizer = optim.Adam(lstm_model.parameters())
loss_fn = nn.CrossEntropyLoss(reduction="sum")
loader_train = datatools.DataLoader(datatools.TensorDataset(X_train, y_train), shuffle=True, batch_size=batch_size)
loader_test = datatools.DataLoader(datatools.TensorDataset(X_test, y_test), shuffle=False, batch_size=batch_size)

In [ ]:
best_model = None
best_loss = np.inf
loss_train = []
loss_test = []
for epoch in range(n_epochs):
    lstm_model.train()
    loss_tr =0
    for X_batch, y_batch in loader_train:
        X_batch = X_batch.to(device)
        y_batch = y_batch.to(device)
        y_pred = lstm_model(X_batch)
        loss_tr = loss_fn(y_pred, y_batch)
        optimizer.zero_grad()
        loss_tr.backward()
        optimizer.step()
        loss_tr += loss
    loss_train.append(loss_tr)
    print('Epoch %s/%s: Train Cross-entropy: %.6f' % (epoch+1,n_epochs, loss_tr.item()/len(loader_train)))
        
    # Validation
    lstm_model.eval()
    loss= 0
    with torch.no_grad():
        for X_batch_test, y_batch_test in loader_test:
            X_batch_test = X_batch_test.to(device)
            y_batch_test = y_batch_test.to(device)
            y_pred_test = lstm_model(X_batch_test)
            loss += loss_fn(y_pred_test, y_batch_test)
            # print( loss)
            # loss_te += loss
        loss_test.append(loss)
        if loss < best_loss:
            best_loss = loss
            best_model = lstm_model.state_dict()
        
        print("Epoch %s/%s: Test Cross-entropy: %.6f" % (epoch+1,n_epochs, loss/len(loader_test)))



In [ ]:
torch.save(best_model, "best_model_lstm.pth")

In [ ]:
loss_train_to_plot = []
for i in loss_train:
    loss_train_to_plot.append(i.item())
loss_test_to_plot = []
for i in loss_test:
    loss_test_to_plot.append(i.item())
losses = {'Training Loss': loss_train_to_plot, 'Validation Loss': loss_test_to_plot}
plot_training(losses)

In [ ]:
device = 'mps'
lstm_best = LSTM_Model().to(device) 
lstm_best.load_state_dict(best_model)

batch_idx = np.random.randint(0, train_batches, )
series_idx = np.random.randint(0, stocks, )
batch= data[batch_idx]
batch= np.transpose(batch)
x = torch.from_numpy(batch)
vq_vae_conv2.eval()
x = x.view( x.size(0), 1, x.size(1))
# print(x.shape)
x_recon, loss, perplexity, encodings, encoding_indices, quantized  = vq_vae_conv2(x)
x_recon_sample = x_recon[series_idx].detach().numpy()
x_original = x[series_idx].detach().numpy()
encoding_indices_sample = encoding_indices[series_idx].detach().numpy()

In [ ]:
lstm_best.eval()
series = encoding_indices_sample[0:seq_length]
out_sample = encoding_indices_sample[seq_length:]
print(series.shape)
print(out_sample.shape)

In [ ]:
encodings_pred_indices = []
with torch.no_grad():
    for i in range(lstm_data.shape[1]):
        x_sample = series / float(num_embeddings)
        x_sample = torch.tensor(x_sample, dtype=torch.float32).reshape(1, seq_length, 1)
        x_sample = x_sample.to(device)
        prediction = lstm_best(x_sample)
        encoding_index = prediction.argmax()
        series = np.append(series, encoding_index.cpu().numpy())
        encodings_pred_indices.append(encoding_index.cpu().numpy())
        series = series[1:]
print()
print("Done.")

In [ ]:
encodings_pred_indices = np.array(encodings_pred_indices)
encodings_pred_indices

out_sample
len(out_sample)
num_embeddings
# print(mean_squared_error(encodings_pred_indices[:seq_length], out_sample))
# encodings_pred =np.expand_dims(encodings_pred, 1)
encodings_pred_indices= torch.from_numpy(encodings_pred_indices)
encodings_pred_indices = encodings_pred_indices.view(-1,1)


encodings_pred_indices.shape


In [ ]:

encodings_pred = torch.zeros(encodings_pred_indices.shape[0], num_embeddings, device='cpu').double()
print(encodings_pred.shape)


encodings_pred.scatter_(1,encodings_pred_indices, 1)
print(encodings_pred.shape)
embeddings =vq_vae_conv2._vq_vae._embedding 
# embeddings = embeddings.weight.detach().cpu().numpy()
print('embeddigs',embeddings.weight.detach().cpu().numpy().shape)
print('encode shape',z.shape)
quantized_pred = torch.matmul(encodings_pred, embeddings.weight).view(z.shape)
print('Quantized shape: ', quantized_pred.shape)
x_gen = vq_vae_conv2.decoder(quantized_pred)
x_gen.shape

(434, 256)

In [ ]:
x_gen = x_gen.detach().numpy()
x_gen = x_gen.reshape(-1)
print(x_gen.shape)
plt.plot(x_gen)
plt.show() 